In [1]:
First_name = "Clement"
Last_Name = "Bannem"

# Libraries

In [2]:
library(ROCR)
# if(!require("Boruta"))install.packages("Boruta")
library(Boruta)
library(stringr)          # String, text processing

# Machine learning library
library(mlr)          # Machine learning framework
library(caret)         # Data processing and machine learning framework
library(MASS)          # LDA
library(randomForest)  # RF
library(gbm)           # Boosting Tree
library(xgboost)       # XGboost
library(caret)

library(tidyverse)  #Data manipulation and visualization

# install.packages("dummies")
library(dummies)

Warning message:
"package 'ROCR' was built under R version 3.6.2"Loading required package: gplots
Warning message:
"package 'gplots' was built under R version 3.6.2"
Attaching package: 'gplots'

The following object is masked from 'package:stats':

    lowess

Warning message:
"package 'Boruta' was built under R version 3.6.3"Loading required package: ranger
Warning message:
"package 'ranger' was built under R version 3.6.3"Warning message:
"package 'stringr' was built under R version 3.6.1"Warning message:
"package 'mlr' was built under R version 3.6.2"Loading required package: ParamHelpers
Warning message:
"package 'ParamHelpers' was built under R version 3.6.2"'mlr' is in maintenance mode since July 2019. Future development
efforts will go into its successor 'mlr3' (<https://mlr3.mlr-org.com>).

Attaching package: 'mlr'

The following object is masked from 'package:ROCR':

    performance

Warning message:
"package 'caret' was built under R version 3.6.2"Loading required package: la

## Data Processing
### DataSummary

In [3]:
train_data = read.csv(file = "C:/Users/abannem/Documents/MSc In Big Data For Business/Courses/16. Statistical & Machine Learning Approches For Marketing/Individual Project/In-class Kaggle Competition/data/bank_mkt/bank_mkt_train.csv")
test_data = read.csv(file = "C:/Users/abannem/Documents/MSc In Big Data For Business/Courses/16. Statistical & Machine Learning Approches For Marketing/Individual Project/In-class Kaggle Competition/data/bank_mkt/bank_mkt_test.csv")

In [4]:
# Checking of data type

str(train_data)

'data.frame':	7000 obs. of  21 variables:
 $ client_id     : int  2 3 4 5 6 7 8 9 14 15 ...
 $ age           : int  29 39 49 32 29 51 34 52 52 29 ...
 $ job           : Factor w/ 12 levels "admin.","blue-collar",..: 4 11 2 7 1 7 2 8 1 1 ...
 $ marital       : Factor w/ 4 levels "divorced","married",..: 3 2 2 3 3 2 2 2 2 3 ...
 $ education     : Factor w/ 8 levels "basic.4y","basic.6y",..: 4 3 2 7 4 7 1 4 7 7 ...
 $ default       : Factor w/ 2 levels "no","unknown": 1 2 2 1 2 2 1 1 1 1 ...
 $ housing       : Factor w/ 3 levels "no","unknown",..: 1 3 1 3 3 3 3 3 3 3 ...
 $ loan          : Factor w/ 3 levels "no","unknown",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ contact       : Factor w/ 2 levels "cellular","telephone": 2 2 1 1 1 2 1 1 1 1 ...
 $ month         : Factor w/ 10 levels "apr","aug","dec",..: 7 5 8 7 4 5 8 8 8 5 ...
 $ day_of_week   : Factor w/ 5 levels "fri","mon","thu",..: 2 1 4 2 1 4 4 4 3 2 ...
 $ campaign      : int  3 6 2 3 2 1 1 1 3 1 ...
 $ pdays         : int  999 999 999 999 9

In [5]:
head(train_data)

client_id,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,subscribe
2,29,housemaid,single,high.school,no,no,no,telephone,may,...,3,999,0,nonexistent,1.1,93.994,-36.4,4.858,5191.0,0
3,39,unemployed,married,basic.9y,unknown,yes,no,telephone,jun,...,6,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,0
4,49,blue-collar,married,basic.6y,unknown,no,no,cellular,nov,...,2,999,0,nonexistent,-0.1,93.200,-42.0,4.153,5195.8,0
5,32,self-employed,single,university.degree,no,yes,no,cellular,may,...,3,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,0
6,29,admin.,single,high.school,unknown,yes,no,cellular,jul,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,0
7,51,self-employed,married,university.degree,unknown,yes,no,telephone,jun,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.864,5228.1,0


###### Splitting into training and test set

In [6]:
set.seed(1)
smp_size <- floor(0.75 * nrow(train_data))

# set the seed to make your partition reproducible
set.seed(1)
train_ind <- sample(seq_len(nrow(train_data)), size = smp_size)

train <- train_data[train_ind, ]
test <- train_data[-train_ind, ]

In [7]:
# set.seed(1)

# train_idx <- caret::createDataPartition(y=train_data[, 'subscribe'], p=.70, list=F)

# train <- train_data[train_idx, ]  # Train 60%
# test <- train_data[-train_idx, ]  # Test 40%

In [8]:
# Check the train and test set distribution

table(train$subscribe)
table(test$subscribe)


   0    1 
4645  605 


   0    1 
1533  217 

In [9]:
# str(train)

### Features ingineering

In [10]:
# Get the IV and DV list name
# Dependent variable (DV)
dv_list <- c('subscribe')
# Independent variable (IV)
iv_list <- setdiff(colnames(train), dv_list)  # Exclude the target variable
iv_list <- setdiff(iv_list, 'client_id')  # Exclude the client_id

In [11]:
iv_bool_list <- c()  # List to store boolean variable
iv_cat_list <- c()  # List to store categorical variable

for (v in iv_list) {
    if (class(train[, v]) == 'logical') {  # Logical == boolean variable
        iv_bool_list <- c(iv_bool_list, v)
    }
    else if (class(train[, v]) == 'factor') {  # Factor == categorical variable
        iv_cat_list <- c(iv_cat_list, v)
    }
}

In [12]:
iv_cat_list

[1] "job"         "marital"     "education"   "default"     "housing"    
 [6] "loan"        "contact"     "month"       "day_of_week" "poutcome"

In [13]:
# Dummytizing of all the categorical variables on the train set

train <- dummy.data.frame(train, names = iv_cat_list, sep = ".")

Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument 

In [14]:
train["education.illiterate"] = NULL

In [15]:
# Dummytizing of all the categorical variables on the test set

test <- dummy.data.frame(test, names = iv_cat_list, sep = ".")

Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument 

In [16]:
# Dummytizing of all the categorical variables on the test data set

test_data <- dummy.data.frame(test_data, names = iv_cat_list, sep = ".")

Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument 

In [17]:
# str(test)

###### binning of the variable "age"

In [18]:
bins <- c(-Inf, 40, 60, 80, Inf)

In [19]:
values <- c(1, 2, 3, 4)

In [20]:
# below 40 = 1
# from 40 to 60 = 2
# from 60 to 80 = 3
# above 80 = 4

train$age.binned <- as.numeric(cut(train$age, breaks = bins, labels = values))
test$age.binned <- as.numeric(cut(test$age, breaks = bins, labels = values))

test_data$age.binned <- as.numeric(cut(test_data$age, breaks = bins, labels = values))

In [21]:
# DataSet$age.binned <- as.numeric(DataSet$age.binned)

##### pdays

In [22]:
# Add new variable to train and test (holdout)
# pdays == 999 is a special value
# Train, valid, test
train[, 'pdays_999'] <- as.logical(train$pdays == 999)
test[, 'pdays_999'] <- as.logical(test$pdays == 999)
# Test (holdout)
test_data[, 'pdays_999'] <- as.logical(test_data$pdays == 999)

In [23]:
# Convert boolean to int
for (v in iv_bool_list) {
    # Train, valid, test
    train[, v] <- as.integer(train[, v])
    test[, v] <- as.integer(test[, v])
    
    # Test (holdout)
    test_data[, v] <- as.integer(test_data[, v])
}

In [24]:
# str(train)

##### Variable Correcting and filtering

In [25]:
# Check and correct infinite values on the train and test set

sum(apply(sapply(train, is.infinite), 2, sum))
sum(apply(sapply(test, is.infinite), 2, sum))

sum(apply(sapply(test_data, is.infinite), 2, sum))

[1] 0

[1] 0

[1] 0

In [26]:
# Check and correct missing values

sum(apply(is.na(train), 2, sum))
sum(apply(is.na(test), 2, sum))

sum(apply(is.na(test_data), 2, sum))

[1] 0

[1] 0

[1] 0

### Variable Selection : Boruta method

In [27]:
# Perform Boruta search
boruta_output <- Boruta(subscribe + client_id~., data=train, doTrace=0)

In [28]:
names(boruta_output)

[1] "finalDecision" "ImpHistory"    "pValue"        "maxRuns"      
 [5] "light"         "mcAdj"         "timeTaken"     "roughfixed"   
 [9] "call"          "impSource"

In [29]:
# Get significant variables including tentatives
boruta_signif <- getSelectedAttributes(boruta_output, withTentative = TRUE)

In [30]:
print(boruta_signif)

 [1] "age"               "job.unknown"       "contact.cellular" 
 [4] "contact.telephone" "month.aug"         "pdays"            
 [7] "previous"          "poutcome.success"  "emp.var.rate"     
[10] "cons.price.idx"    "euribor3m"         "nr.employed"      
[13] "age.binned"       


In [31]:
# boruta_signif <- boruta_signif[c(1:10)]

In [32]:
# print(boruta_signif)

In [33]:
# Apply variable selection to the data
# Train
var_select <- names(train)[names(train) %in% boruta_signif]
train_processed <- train[, c('client_id', var_select, 'subscribe')]

In [34]:
# Test
var_select <- names(test)[names(test) %in% boruta_signif]
test_processed <- test[, c('client_id', var_select, 'subscribe')]

In [35]:
# Test (holdout)
var_select <- names(test_data)[names(test_data) %in% boruta_signif]
test_holdout_processed <- test_data[, c('client_id', var_select)]

In [36]:
dim(train_processed)
dim(test_processed)
dim(test_holdout_processed)

[1] 5250   15

[1] 1750   15

[1] 3000   14

In [37]:
# Rename the data columns
for (v in colnames(train_processed)) {
    
    # Fix the column name
    fix_name <- str_replace_all(v, "[^[:alnum:] ]", "_")
    fix_name <- gsub(' +', '', fix_name) 
    
    # Train, valid, test
    colnames(train_processed)[colnames(train_processed) == v] <- fix_name
    colnames(test_processed)[colnames(test_processed) == v] <- fix_name
    
    # Test (holdout)
    colnames(test_holdout_processed)[colnames(test_holdout_processed) == v] <- fix_name
    
}

In [38]:
head(train_processed)

,client_id,age,job_unknown,contact_cellular,contact_telephone,month_aug,pdays,previous,poutcome_success,emp_var_rate,cons_price_idx,euribor3m,nr_employed,age_binned,subscribe
1017,1472,44,0,1,0,0,999,0,0,1.4,93.918,4.960,5228.1,2,0
4775,6809,35,0,0,1,0,999,0,0,-1.8,92.893,1.281,5099.1,1,0
2177,3103,35,1,0,1,0,999,0,0,1.1,93.994,4.855,5191.0,1,0
5026,7194,44,0,1,0,0,999,0,0,-1.8,92.893,1.299,5099.1,2,0
1533,2195,32,0,0,1,0,999,0,0,1.4,93.918,4.968,5228.1,1,0
4567,6519,53,0,1,0,0,999,0,0,-1.8,93.075,1.479,5099.1,2,0


In [39]:
# Output predicted file

write.csv(test_data, 'FinalBaseTable.csv', row.names=FALSE)

### 1 - Logistic Regression

In [40]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=5, predict="both")

# Define the model
learner <- makeLearner("classif.logreg", predict.type="prob", fix.factors.prediction=T)

# Define the task
train_task <- makeClassifTask(id="bank_train", data=train_processed[, -1], target="subscribe")

# Set hyper parameter tuning
tune_params <- makeParamSet(
)
ctrl = makeTuneControlGrid()

# Run the hyper parameter tuning with k-fold CV
if (length(tune_params$pars) > 0) {
    # Run parameter tuning
    res <- tuneParams(learner, task=train_task, resampling=rdesc,
      par.set=tune_params, control=ctrl, measures=list(mlr::auc))
    
    # Extract best model
    best_learner <- res$learner
    
} else {
    # Simple cross-validation
    res <- resample(learner, train_task, rdesc, measures=list(mlr::auc, setAggregation(mlr::auc, train.mean)))
    
    # No parameter for tuning, only 1 best learner
    best_learner <- learner
}

Resampling: cross-validation
Measures:             auc.train   auc.test    
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"[Resample] iter 1:    0.7854166   0.7873739   
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"[Resample] iter 2:    0.7920924   0.7529193   
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-defi

In [41]:
# Retrain the model with tbe best hyper-parameters
best_md <- mlr::train(best_learner, train_task)

In [42]:
# Make prediction on test data
pred <- predict(best_md, newdata=test_processed[, -1])
performance(pred, measures=mlr::auc)

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"

auc 
0.8010287

In [43]:
# Make prediction on test (holdout) data
pred <- predict(best_md, newdata=test_holdout_processed[, -1])
pred

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"

Prediction: 3000 observations
predict.type: prob
threshold: 0=0.50,1=0.50
time: 0.02
     prob.0     prob.1 response
1 0.9028804 0.09711956        0
2 0.9516697 0.04833025        0
3 0.8597326 0.14026743        0
4 0.9749095 0.02509051        0
5 0.9560173 0.04398269        0
6 0.9483242 0.05167577        0
... (#rows: 3000, #cols: 3)

In [44]:
# Output predicted file
output <- data.frame(client_id=test_data$client_id, subscribe=pred$data$prob.1)
write.csv(output, 'logreg.csv', row.names=FALSE)

### 2 - Random Forest Model

In [45]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=10)

# Define the model
learner <- makeLearner("classif.randomForest", predict.type="prob", fix.factors.prediction=T)

# Define the task
train_task <- makeClassifTask(id="bank_train", data=train_processed[, -1], target="subscribe")

# Set hyper parameter tuning
tune_params <- makeParamSet(
      makeIntegerParam('ntree', lower = 50, upper = 500),
      makeIntegerParam('mtry', lower = 3, upper = 10),
      makeIntegerParam('nodesize', lower = 10, upper = 50)
)
ctrl = makeTuneControlRandom(maxit = 10L)

# Run the hyper parameter tuning with k-fold CV
if (length(tune_params$pars) > 0) {
    # Run parameter tuning
    res <- tuneParams(learner, task=train_task, resampling=rdesc,
      par.set=tune_params, control=ctrl, measures=list(mlr::auc))
    
    # Extract best model
    best_learner <- res$learner
    
} else {
    # Simple cross-validation
    res <- resample(learner, train_task, rdesc, measures=list(mlr::auc))
    
    # No parameter for tuning, only 1 best learner
    best_learner <- learner
}

[Tune] Started tuning learner classif.randomForest for parameter set:
            Type len Def    Constr Req Tunable Trafo
ntree    integer   -   - 50 to 500   -    TRUE     -
mtry     integer   -   -   3 to 10   -    TRUE     -
nodesize integer   -   -  10 to 50   -    TRUE     -
With control class: TuneControlRandom
Imputation value: -0
[Tune-x] 1: ntree=258; mtry=7; nodesize=48
[Tune-y] 1: auc.test.mean=0.7610168; time: 0.2 min
[Tune-x] 2: ntree=278; mtry=7; nodesize=17
[Tune-y] 2: auc.test.mean=0.7652011; time: 0.2 min
[Tune-x] 3: ntree=182; mtry=5; nodesize=36
[Tune-y] 3: auc.test.mean=0.7626760; time: 0.1 min
[Tune-x] 4: ntree=138; mtry=6; nodesize=16
[Tune-y] 4: auc.test.mean=0.7582768; time: 0.1 min
[Tune-x] 5: ntree=281; mtry=6; nodesize=38
[Tune-y] 5: auc.test.mean=0.7611588; time: 0.2 min
[Tune-x] 6: ntree=121; mtry=7; nodesize=36
[Tune-y] 6: auc.test.mean=0.7607376; time: 0.1 min
[Tune-x] 7: ntree=310; mtry=7; nodesize=36
[Tune-y] 7: auc.test.mean=0.7663135; time: 0.2 min
[

In [46]:
# Retrain the model with tbe best hyper-parameters
best_md <- mlr::train(best_learner, train_task)

In [47]:
# Make prediction on test data
pred <- predict(best_md, newdata=test_processed[, -1])
performance(pred, measures=mlr::auc)

auc 
0.7847388

In [48]:
# Make prediction on test data
pred <- predict(best_md, newdata=test_holdout_processed[, -1])
pred

Prediction: 3000 observations
predict.type: prob
threshold: 0=0.50,1=0.50
time: 0.09
  prob.0 prob.1 response
1  1.000  0.000        0
2  1.000  0.000        0
3  0.686  0.314        0
4  0.998  0.002        0
5  0.994  0.006        0
6  1.000  0.000        0
... (#rows: 3000, #cols: 3)

In [49]:
# Output predicted file
output <- data.frame(client_id=test_data$client_id, subscribe=pred$data$prob.1)
write.csv(output, 'randomforest.csv', row.names=FALSE)

### 3 - Xtreme Gradient Boosting

In [50]:
str(train_processed)

'data.frame':	5250 obs. of  15 variables:
 $ client_id        : int  1472 6809 3103 7194 2195 6519 3340 384 5774 7593 ...
 $ age              : int  44 35 35 44 32 53 28 51 36 62 ...
 $ job_unknown      : int  0 0 1 0 0 0 0 0 0 0 ...
 $ contact_cellular : int  1 0 0 1 0 1 1 1 1 1 ...
 $ contact_telephone: int  0 1 1 0 1 0 0 0 0 0 ...
 $ month_aug        : int  0 0 0 0 0 0 0 0 1 1 ...
 $ pdays            : int  999 999 999 999 999 999 999 999 999 999 ...
 $ previous         : int  0 0 0 0 0 0 0 0 0 1 ...
 $ poutcome_success : int  0 0 0 0 0 0 0 0 0 0 ...
 $ emp_var_rate     : num  1.4 -1.8 1.1 -1.8 1.4 -1.8 -1.8 -1.7 1.4 -2.9 ...
 $ cons_price_idx   : num  93.9 92.9 94 92.9 93.9 ...
 $ euribor3m        : num  4.96 1.28 4.86 1.3 4.97 ...
 $ nr_employed      : num  5228 5099 5191 5099 5228 ...
 $ age_binned       : num  2 1 1 2 1 2 1 2 1 3 ...
 $ subscribe        : int  0 0 0 0 0 0 0 0 0 0 ...


In [51]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=10, predict="both")

# Define the model
learner <- makeLearner("classif.xgboost", predict.type="prob")

# Define the task
train_task <- makeClassifTask(id="bank_train", data=train_processed[, -1], target="subscribe")

# Set hyper parameter tuning
tune_params <- makeParamSet(
    
    makeIntegerParam("nrounds", lower = 200, upper = 600),
    makeIntegerParam("max_depth", lower = 3, upper = 20),
    makeNumericParam("lambda", lower = 0.55, upper = 0.60),
    makeNumericParam("eta", lower = 0.001, upper = 0.5),
    makeNumericParam("subsample", lower = 0.10, upper = 0.80),
    makeNumericParam("min_child_weight", lower = 0.10, upper = 5),
    makeNumericParam("colsample_bytree", lower = 0.2, upper = 0.8)
    
)
ctrl = makeTuneControlRandom(maxit = 100L)

# Run the hyper parameter tuning with k-fold CV
if (length(tune_params$pars) > 0) {
    # Run parameter tuning
    res <- tuneParams(learner, task=train_task, resampling=rdesc,
      par.set=tune_params, control=ctrl, measures=list(mlr::auc))
    
    # Extract best model
    best_learner <- res$learner
    
} else {
    # Simple cross-validation
    res <- resample(learner, train_task, rdesc, measures=list(mlr::auc, setAggregation(mlr::auc, train.mean)))
    
    # No parameter for tuning, only 1 best learner
    best_learner <- learner
}

[Tune] Started tuning learner classif.xgboost for parameter set:
                    Type len Def       Constr Req Tunable Trafo
nrounds          integer   -   -   200 to 600   -    TRUE     -
max_depth        integer   -   -      3 to 20   -    TRUE     -
lambda           numeric   -   -  0.55 to 0.6   -    TRUE     -
eta              numeric   -   - 0.001 to 0.5   -    TRUE     -
subsample        numeric   -   -   0.1 to 0.8   -    TRUE     -
min_child_weight numeric   -   -     0.1 to 5   -    TRUE     -
colsample_bytree numeric   -   -   0.2 to 0.8   -    TRUE     -
With control class: TuneControlRandom
Imputation value: -0
[Tune-x] 1: nrounds=543; max_depth=14; lambda=0.586; eta=0.291; subsample=0.797; min_child_weight=4.16; colsample_bytree=0.325
[Tune-y] 1: auc.test.mean=0.6996991; time: 0.4 min
[Tune-x] 2: nrounds=481; max_depth=6; lambda=0.579; eta=0.49; subsample=0.146; min_child_weight=1.74; colsample_bytree=0.579
[Tune-y] 2: auc.test.mean=0.6658887; time: 0.3 min
[Tune-x] 3

[Tune-y] 43: auc.test.mean=0.6978201; time: 0.2 min
[Tune-x] 44: nrounds=579; max_depth=8; lambda=0.588; eta=0.0941; subsample=0.174; min_child_weight=4.69; colsample_bytree=0.565
[Tune-y] 44: auc.test.mean=0.7633367; time: 0.4 min
[Tune-x] 45: nrounds=344; max_depth=7; lambda=0.586; eta=0.0861; subsample=0.494; min_child_weight=4.6; colsample_bytree=0.53
[Tune-y] 45: auc.test.mean=0.7571901; time: 0.2 min
[Tune-x] 46: nrounds=274; max_depth=16; lambda=0.578; eta=0.0443; subsample=0.448; min_child_weight=3.9; colsample_bytree=0.495
[Tune-y] 46: auc.test.mean=0.7685038; time: 0.3 min
[Tune-x] 47: nrounds=577; max_depth=15; lambda=0.589; eta=0.26; subsample=0.503; min_child_weight=2.41; colsample_bytree=0.526
[Tune-y] 47: auc.test.mean=0.6878760; time: 0.6 min
[Tune-x] 48: nrounds=586; max_depth=18; lambda=0.563; eta=0.197; subsample=0.26; min_child_weight=1.36; colsample_bytree=0.373
[Tune-y] 48: auc.test.mean=0.7088438; time: 0.5 min
[Tune-x] 49: nrounds=246; max_depth=6; lambda=0.566;

[Tune-y] 89: auc.test.mean=0.7379537; time: 0.6 min
[Tune-x] 90: nrounds=423; max_depth=18; lambda=0.571; eta=0.386; subsample=0.629; min_child_weight=2.47; colsample_bytree=0.225
[Tune-y] 90: auc.test.mean=0.7137530; time: 0.3 min
[Tune-x] 91: nrounds=542; max_depth=15; lambda=0.555; eta=0.321; subsample=0.763; min_child_weight=0.289; colsample_bytree=0.641
[Tune-y] 91: auc.test.mean=0.6949668; time: 0.7 min
[Tune-x] 92: nrounds=450; max_depth=8; lambda=0.567; eta=0.0641; subsample=0.297; min_child_weight=3.22; colsample_bytree=0.706
[Tune-y] 92: auc.test.mean=0.7444792; time: 0.4 min
[Tune-x] 93: nrounds=352; max_depth=18; lambda=0.584; eta=0.388; subsample=0.252; min_child_weight=3.65; colsample_bytree=0.643
[Tune-y] 93: auc.test.mean=0.7029743; time: 0.3 min
[Tune-x] 94: nrounds=445; max_depth=11; lambda=0.6; eta=0.414; subsample=0.284; min_child_weight=4.83; colsample_bytree=0.445
[Tune-y] 94: auc.test.mean=0.7206044; time: 0.3 min
[Tune-x] 95: nrounds=301; max_depth=20; lambda=0.

In [52]:
# Retrain the model with tbe best hyper-parameters
best_md <- mlr::train(best_learner, train_task)

In [57]:
# Make prediction on test data
pred <- predict(best_md, newdata=test_processed[, -1])
performance(pred, measures=mlr::auc)

auc 
0.7763654

In [58]:
# Make prediction on test data
pred <- predict(best_md, newdata=test_holdout_processed[, -1])
pred

Prediction: 3000 observations
predict.type: prob
threshold: 0=0.50,1=0.50
time: 0.00
     prob.1    prob.0 response
1 0.3682989 0.6317011        0
2 0.3682989 0.6317011        0
3 0.4304538 0.5695462        0
4 0.3682989 0.6317011        0
5 0.3682989 0.6317011        0
6 0.3682989 0.6317011        0
... (#rows: 3000, #cols: 3)

In [59]:
# Output predicted file
output <- data.frame(client_id=test_data$client_id, subscribe=pred$data$prob.1)
write.csv(output, 'xgboost.csv', row.names=FALSE)

### 4 - Logit Leaf Model

Reference:  

De Caigny, A., Coussement, K., & De Bock, K. W. (2018). A new hybrid classification algorithm for customer churn prediction based on logistic regression and decision trees. European Journal of Operational Research, 269(2), 760-772.

In [60]:
# Define the LLM model and its parameters
makeRLearner.classif.llm = function() {
  makeRLearnerClassif(
    cl = "classif.llm",
    package = "LLM",
    par.set = makeParamSet(
      makeNumericLearnerParam(id="threshold_pruning", default=0.25, tunable=T),
      makeNumericLearnerParam(id="nbr_obs_leaf", default=100, tunable=T)
    ),
    properties = c("twoclass", "numerics", "prob"),
    name = "Logit Leaf Model",
    short.name = "llm",
    note = "Logit Leaf Model Classifier for Binary Classification."
  )
}

# Define the train function for LLM
trainLearner.classif.llm = function(.learner, .task, .subset, .weights=NULL, ...) {
  
  # Prepare data to train the LLM model
  data = getTaskData(.task, .subset)
  X <- data[, getTaskFeatureNames(.task)]
  Y <- data[, getTaskTargetNames(.task)]
  
  # Train the LLM model
  LLM::llm(X, Y, ...)
}

# Define the prediction function for LLM
predictLearner.classif.llm = function(.learner, .model, .newdata, ...) {
  
  # Prepare test data for making prediction
  X <- .newdata[, setdiff(names(.newdata), c(.model$task.desc$target))]
  
  # Make prediction using LLM model
  p = LLM::predict.llm(.model$learner.model, X)
  out <- data.frame(Y=p$probability, N=1-p$probability)
  
  return(as.matrix(out))
}

In [61]:
# Fix the target variable from [0, 1] to [Y, N]
train_processed['subscribe'] = as.factor(ifelse(train_processed['subscribe'] == 1, 'Y', 'N'))
test_processed['subscribe'] = as.factor(ifelse(test_processed['subscribe'] == 1, 'Y', 'N'))

In [62]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=10)

# Define the model
learner <- makeLearner("classif.llm", predict.type="prob", fix.factors.prediction=T)

# Define the task
train_task <- makeClassifTask(id="bank_train", data=train_processed[, -1], target="subscribe")

# Set hyper parameter tuning
tune_params <- makeParamSet(
  makeDiscreteParam("threshold_pruning", values=c(0.01, 0.10, 0.15, 0.20, 0.25, 0.30)),
  makeDiscreteParam("nbr_obs_leaf", values=round(nrow(train_processed) * c(0.01, 0.025, 0.05, 0.1, 0.25, 0.5)))
)
ctrl = makeTuneControlGrid()

# Run the hyper parameter tuning with k-fold CV
if (length(tune_params$pars) > 0) {
    # Run parameter tuning
    res <- tuneParams(learner, task=train_task, resampling=rdesc,
      par.set=tune_params, control=ctrl, measures=list(mlr::auc))
    
    # Extract best model
    best_learner <- res$learner
    
} else {
    # Simple cross-validation
    res <- resample(learner, train_task, rdesc, measures=list(mlr::auc))
    
    # No parameter for tuning, only 1 best learner
    best_learner <- learner
}

[Tune] Started tuning learner classif.llm for parameter set:
                      Type len Def                     Constr Req Tunable Trafo
threshold_pruning discrete   -   - 0.01,0.1,0.15,0.2,0.25,0.3   -    TRUE     -
nbr_obs_leaf      discrete   -   -   52,131,262,525,1312,2625   -    TRUE     -
With control class: TuneControlGrid
Imputation value: -0
[Tune-x] 1: threshold_pruning=0.01; nbr_obs_leaf=52
[Tune-y] 1: auc.test.mean=0.7802661; time: 0.3 min
[Tune-x] 2: threshold_pruning=0.1; nbr_obs_leaf=52
[Tune-y] 2: auc.test.mean=0.7790630; time: 0.3 min
[Tune-x] 3: threshold_pruning=0.15; nbr_obs_leaf=52
[Tune-y] 3: auc.test.mean=0.7783884; time: 0.3 min
[Tune-x] 4: threshold_pruning=0.2; nbr_obs_leaf=52
[Tune-y] 4: auc.test.mean=0.7740686; time: 0.3 min
[Tune-x] 5: threshold_pruning=0.25; nbr_obs_leaf=52
[Tune-y] 5: auc.test.mean=0.7706940; time: 0.3 min
[Tune-x] 6: threshold_pruning=0.3; nbr_obs_leaf=52
[Tune-y] 6: auc.test.mean=0.7662159; time: 0.3 min
[Tune-x] 7: threshold_pruni

In [69]:
# Retrain the model with tbe best hyper-parameters
best_md <- mlr::train(best_learner, train_task)

In [70]:
# Make prediction on test data
pred <- predict(best_md, newdata=test_processed[, -1])
performance(pred, measures=mlr::auc)

auc 
0.801433

In [71]:
# Make prediction on test data
pred <- predict(best_md, newdata=test_holdout_processed[, -1])
pred

Prediction: 3000 observations
predict.type: prob
threshold: N=0.50,Y=0.50
time: 0.00
      prob.Y    prob.N response
1 0.10183594 0.8981641        N
2 0.04835625 0.9516438        N
3 0.13714634 0.8628537        N
4 0.02275134 0.9772487        N
5 0.04436840 0.9556316        N
6 0.05463644 0.9453636        N
... (#rows: 3000, #cols: 3)

In [78]:
# Output predicted file
output <- data.frame(client_id=test_data$client_id, subscribe=pred$data$prob.1)
write.csv(output, 'llm.csv', row.names=FALSE)

ERROR: Error in data.frame(client_id = test_holdout_processed$client_id, subscribe = pred$data$prob.1): arguments imply differing number of rows: 3000, 0


### 5 - Naive Bayes

In [73]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=10)

# Define the model
learner <- makeLearner("classif.naiveBayes", predict.type="prob", fix.factors.prediction=T)

# Define the task
train_task <- makeClassifTask(id="bank_train", data=train_processed[, -1], target="subscribe")

# Set hyper parameter tuning
tune_params <- makeParamSet(
    makeDiscreteParam("laplace", values=c(0, 5, by = 0.5))
)
ctrl = makeTuneControlGrid()

# Run the hyper parameter tuning with k-fold CV
if (length(tune_params$pars) > 0) {
    # Run parameter tuning
    res <- tuneParams(learner, task=train_task, resampling=rdesc,
      par.set=tune_params, control=ctrl, measures=list(mlr::auc))
    
    # Extract best model
    best_learner <- res$learner
    
} else {
    # Simple cross-validation
    res <- resample(learner, train_task, rdesc, measures=list(mlr::auc))
    
    # No parameter for tuning, only 1 best learner
    best_learner <- learner
}

[Tune] Started tuning learner classif.naiveBayes for parameter set:
            Type len Def Constr Req Tunable Trafo
laplace discrete   -   - 0,5,by   -    TRUE     -
With control class: TuneControlGrid
Imputation value: -0
[Tune-x] 1: laplace=0
[Tune-y] 1: auc.test.mean=0.7685295; time: 0.0 min
[Tune-x] 2: laplace=5
[Tune-y] 2: auc.test.mean=0.7685295; time: 0.0 min
[Tune-x] 3: laplace=by
[Tune-y] 3: auc.test.mean=0.7685295; time: 0.0 min
[Tune] Result: laplace=5 : auc.test.mean=0.7685295


In [74]:
# Retrain the model with tbe best hyper-parameters
best_md <- mlr::train(best_learner, train_task)

In [75]:
# Make prediction on test data
pred <- predict(best_md, newdata=test_processed[, -1])
performance(pred, measures=mlr::auc)

auc 
0.7745062

In [76]:
# Make prediction on test data
pred <- predict(best_md, newdata=test_holdout_processed[, -1])
pred

Prediction: 3000 observations
predict.type: prob
threshold: N=0.50,Y=0.50
time: 0.71
     prob.N       prob.Y response
1 0.9980282 1.971822e-03        N
2 0.9997267 2.732705e-04        N
3 0.9775675 2.243252e-02        N
4 0.9999950 4.998387e-06        N
5 0.9999906 9.412314e-06        N
6 0.9997048 2.952445e-04        N
... (#rows: 3000, #cols: 3)

In [77]:
# Output predicted file
output <- data.frame(client_id=test_data$client_id, subscribe=pred$data$prob.1)
write.csv(output, 'naiveBayes.csv', row.names=FALSE)

ERROR: Error in data.frame(client_id = test_data$client_id, subscribe = pred$data$prob.1): arguments imply differing number of rows: 3000, 0
